In [1]:
import libpysal as ps
import geopandas as gpd
import splot
import matplotlib.pyplot as plt
import esda
import pandas as pd
import seaborn as sns
import numpy as np
import sqlite3 as sq
import statsmodels.api as sm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from scipy import stats


conn = sq.connect('/home/manuel/Desktop/pROYECTO/database.db')




In [2]:
seleccion = pd.read_sql('SELECT * FROM Seleccionados', conn)

MAPA GENERAL DE INGRESOS PROMEDIO MUNICIPIOS CUNDINAMARCA

In [ ]:
import libpysal as ps
from esda import Moran, Moran_Local

# ESTA ES LA BASE CON QUE SE REALIZA LOS GRAFICOS Y LOS ESTUDIOS DE DEPENDENCIA ESPACIAL
cart = seleccion.groupby(['NOMB_MUN','COD_MPIO','LATITUD','LONGITUD']).agg(MEDIA = ('Total_ingresos','mean')).reset_index()

gdf = gpd.read_file('Mapas/Municipio, Distrito y Area no municipalizada.shp')
gdf = gdf[gdf['Depto'] == 'Cundinamarca']

gdf['MpCodigo'] = pd.to_numeric(gdf['MpCodigo'], errors='coerce')
df_ = gdf.merge(cart,left_on='MpCodigo', right_on='COD_MPIO', how='inner')


# GRAFICO DE LOS INGRESOS PROMEDIOS POR MUNICIPIO
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

gdf.plot(ax=ax, color='white', edgecolor='black')
df_.plot(column='MEDIA', ax=ax, legend=True,edgecolor='black',
         legend_kwds={'label': "MEDIA",
                      'orientation': "vertical"})
plt.title("INGRESOS PROMEDIO CUNDINAMARCA", fontsize=16)
plt.show()

In [ ]:
# CALCULO DE INDICE DE MORAN, AVERIGUAR LA DEPENDENCIA ESPACIAL

# LAS DISTANCIAS SE MIDEN DE ACUERDO AL MOVIMIENTO DE LA REINA (QUEEN)

wq =  ps.weights.Queen.from_dataframe(df_)
wq.transform = 'r'

moran = Moran(df_['MEDIA'],wq)

print(f'El indice de Morgan General es de : {moran.I:.2f}')
print(f'El valor p para el sistema de hipótesis es de {moran.p_sim}')


# COMO EL VALOR P ES DE 0.004 ENTNCES SE RECHAZA H0: I=0 POR LO TANTO EXISTE DEPENDENCIA ESPACIAL

In [ ]:
df_.head()

In [ ]:
# DEPENDENCIA LOCAL (GRAFICAS LISA)

# Calcular el índice de Moran Local para la variable 'MEDIA'
ml = esda.Moran_Local(df_['MEDIA'], wq)

# Acceder a los valores p de la simulación
p_values = ml.p_sim

# Asumiendo que ya tienes el GeoDataFrame df_ con los resultados de Moran's I Local
df_['moran_local'] = ml.p_sim  # Añadir el resultado de Moran's I Local al GeoDataFrame

# Graficar en el mapa
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
gdf.plot(ax=ax, color='white', edgecolor='black')
df_.plot(column='moran_local', ax=ax, legend=True,edgecolor='black',
         legend_kwds={'label': "Moran's I Local",
                      'orientation': "horizontal"})

for idx, row in gdf.iterrows():
    ax.annotate(text=row['MpNombre'], xy=(row.geometry.centroid.x, row.geometry.centroid.y),
                fontsize=8, ha='center', color='black')
    
plt.title("Mapa de Moran's I Local", fontsize=10)
plt.show()